# Endangered-birds detection model training

## Import Libraries

In [14]:
# --- Standard Libraries ---
import os
import warnings
import yaml  # To potentially inspect the data.yaml
import traceback # To print error details
from pathlib import Path
import glob # To find image files in a directory
from collections import defaultdict
import time # To add a small delay if needed

# --- Third-party Libraries ---
from ultralytics import YOLO
import torch
import pandas as pd # Useful for analyzing results

## Configuration

In [15]:
# --- Configuration ---
# Path to the base pre-trained model
base_model_path = 'yolo11n.pt'

# Path to your dataset configuration YAML file
# **MAI Requirement:** Ensure this YAML defines 'train', 'val', and ideally a 'test' split.
data_yaml_path = './datasets/bird_dataset_yolo/data.yaml'
# Example data.yaml structure:
# train: ../train/images
# val: ../valid/images
# test: ../test/images  # Optional but recommended for final evaluation
# nc: 5  # number of classes
# names: ['BirdA', 'BirdB', 'BirdC', 'BirdD', 'EndangeredBirdX'] # Class names

# Training parameters
num_epochs = 100
image_size = 320
batch_size = 32
run_name = 'birds_320_python_run1_noprint' # Updated run name slightly
device_to_use = 'cuda' if torch.cuda.is_available() else 'cpu' # Auto-detect GPU or use CPU

# Evaluation & Robustness Configuration
evaluate_on_test_set = True # Set to True if you have a 'test' split in data.yaml
confidence_threshold = 0.25 # Confidence threshold for evaluation/prediction
iou_threshold = 0.5       # IoU threshold for NMS and mAP calculation

## Helper Function

In [16]:
# --- Helper Functions ---
def check_dataset_yaml(yaml_path):
    """Checks if the dataset YAML file exists and has expected keys (using print)."""
    if not os.path.exists(yaml_path):
        print(f"ERROR: Dataset YAML not found at: {yaml_path}")
        return False, None
    try:
        with open(yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)
        required_keys = ['train', 'val', 'nc', 'names']
        missing_keys = [key for key in required_keys if key not in data_config]
        if missing_keys:
            print(f"ERROR: Dataset YAML ({yaml_path}) is missing required keys: {missing_keys}")
            return False, None
        if 'test' not in data_config:
             print(f"WARNING: Dataset YAML ({yaml_path}) does not define a 'test' split. Final evaluation will use the 'val' split.")
        print(f"Dataset YAML loaded successfully from: {yaml_path}")
        print(f"Number of classes (nc): {data_config.get('nc')}")
        print(f"Class names: {data_config.get('names')}")
        return True, data_config
    except Exception as e:
        print(f"ERROR: Error reading or parsing dataset YAML ({yaml_path}): {e}")
        print("Traceback:")
        traceback.print_exc()
        return False, None

## Execution

In [17]:
print("=============================================")
print(" Starting YOLOv8 Training & Evaluation Script")
print("=============================================")
print(f"Configuration:")
print(f"  Base Model: {base_model_path}")
print(f"  Dataset YAML: {data_yaml_path}")
print(f"  Epochs: {num_epochs}")
print(f"  Image Size: {image_size}")
print(f"  Batch Size: {batch_size}")
print(f"  Run Name: {run_name}")
print(f"  Device: {device_to_use}")
print(f"  Evaluate on Test Set: {evaluate_on_test_set}")
print(f"  Confidence Threshold: {confidence_threshold}")
print(f"  IoU Threshold: {iou_threshold}")
# print(f"  Log File: N/A (Using print statements)") # Removed logging

# **MAI Requirement:** Diagnosing issues before deployment (Pre-computation checks)
print("\n--- Checking Dataset Configuration ---")
valid_yaml, data_config = check_dataset_yaml(data_yaml_path)
if not valid_yaml:
    raise Exception("Exiting due to dataset YAML issues.")

# Check device availability
print("\n--- Checking Device ---")
if device_to_use == 'cuda' and not torch.cuda.is_available():
    print("WARNING: CUDA selected but not available. Switching to CPU.")
    device_to_use = 'cpu'
elif device_to_use != 'cpu' and not isinstance(device_to_use, int) and not device_to_use.startswith('cuda'):
     try:
         device_to_use_int = int(device_to_use)
         print(f"Interpreting device '{device_to_use}' as GPU index {device_to_use_int}.")
         device_to_use = device_to_use_int # Use the integer index
     except ValueError:
         print(f"WARNING: Invalid device specified: {device_to_use}. Defaulting to CPU.")
         device_to_use = 'cpu'
print(f"Using device: {device_to_use}")


training_results = None
best_model_path = None
training_run_dir = None

try:
    # 1. Load the base model
    print("\n--- Loading Base Model ---")
    model = YOLO(base_model_path)
    print(f"Loaded base model: {base_model_path}")

    # 2. Start Training
    # **MAI Requirement:** Monitor training progress (Ultralytics outputs to console).
    print("\n--- Initiating Training ---")
    # Add other relevant parameters from your CLI command if needed
    training_results = model.train(
        data=data_yaml_path,
        imgsz=image_size,
        epochs=num_epochs,
        batch=batch_size,
        name=run_name,
        device=device_to_use,
        exist_ok=False, # Prevent overwriting previous runs unless intended
        # Common parameters for robustness / better training:
        patience=20,
        # workers=8,
        # augment=True,
        # degrees=10.0,
        # fliplr=0.5,
        # close_mosaic=10,
    )

    print("\n--- Training Finished ---")
    # The results object contains paths and potentially final metrics
    training_run_dir = training_results.save_dir # e.g., runs/detect/birds_320_python_run1
    best_model_path = os.path.join(training_run_dir, 'weights/best.pt')
    print(f"Results, weights, and logs saved in: {training_run_dir}")
    print(f"Best model weights (used for evaluation) saved at: {best_model_path}")

    # **MAI Requirement:** Diagnosing issues after training (Analyzing Training Runs)
    print(f"\nTo analyze training curves (loss, mAP), run in your terminal: tensorboard --logdir '{os.path.abspath(training_run_dir)}'")
    print("Check validation curves for overfitting (train loss decreasing, val loss increasing) or underfitting.")


except Exception as e:
    print(f"\nERROR: An error occurred during training: {e}")
    print("Traceback:")
    traceback.print_exc()

 Starting YOLOv8 Training & Evaluation Script
Configuration:
  Base Model: yolo11n.pt
  Dataset YAML: ./datasets/bird_dataset_yolo/data.yaml
  Epochs: 100
  Image Size: 320
  Batch Size: 32
  Run Name: birds_320_python_run1_noprint
  Device: cuda
  Evaluate on Test Set: True
  Confidence Threshold: 0.25
  IoU Threshold: 0.5

--- Checking Dataset Configuration ---
Dataset YAML loaded successfully from: ./datasets/bird_dataset_yolo/data.yaml
Number of classes (nc): 10
Class names: ['Australasian_Bittern', 'Australian_Painted_Snipe', 'Gang-gang_Cockatoo', 'Grey_Falcon', 'Painted_Honeyeater', 'Pilotbird', 'Plains_Wanderer', 'Southern_Whiteface', 'Swift_Parrot', 'White-throated_Needletail']

--- Checking Device ---
Using device: cuda

--- Loading Base Model ---


100%|██████████| 5.35M/5.35M [00:00<00:00, 13.5MB/s]

Loaded base model: yolo11n.pt

--- Initiating Training ---
New https://pypi.org/project/ultralytics/8.3.105 available  Update with 'pip install -U ultralytics'


Ultralytics 8.3.101  Python-3.12.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=./datasets/bird_dataset_yolo/data.yaml, epochs=100, time=None, patience=20, batch=32, imgsz=320, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=birds_320_python_run1_noprint10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning C:\Users\Admin\Documents\FIT5120_MAI_Assignment\datasets\bird_dataset_yolo\labels\train... 937 images, 480 backgrounds, 0 corrupt: 100%|██████████| 1417/1417 [00:01<00:00, 822.62it/s]


...

     75/100      2.69G    0.06546     0.4133     0.8858         29        320: 100%|██████████| 45/45 [00:06<00:00,  7.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  4.99it/s]

                   all        359        239       0.93      0.953      0.971      0.965
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 55, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



75 epochs completed in 0.171 hours.
Optimizer stripped from runs\detect\birds_320_python_run1_noprint10\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\birds_320_python_run1_noprint10\weights\best.pt, 5.4MB

Validating runs\detect\birds_320_python_run1_noprint10\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.55it/s]


                   all        359        239      0.949      0.927      0.981      0.978
  Australasian_Bittern         15         15          1      0.781      0.959      0.959
Australian_Painted_Snipe          9          9       0.86          1      0.984      0.984
    Gang-gang_Cockatoo         80         80       0.93       0.99      0.991       0.97
           Grey_Falcon          9          9      0.985      0.889      0.984      0.984
    Painted_Honeyeater         21         21      0.947      0.952      0.976      0.976
             Pilotbird         13         13          1      0.801       0.99       0.99
       Plains_Wanderer          5          5      0.832          1      0.962      0.962
    Southern_Whiteface         47         47      0.976      0.957      0.983      0.969
          Swift_Parrot         22         22      0.984      0.955      0.989      0.989
White-throated_Needletail         18         18      0.978      0.944      0.992      0.992
Speed: 0.1ms pre

## Testing/Inference

In [20]:
##### --- Configuration ---

# 1. PATH TO YOUR TRAINED MODEL
model_path = './runs/detect/birds_320_python_run1_noprint10/weights/best.pt' # ADJUST THIS

# 2. INPUT SOURCE: Path to the PARENT directory containing class subfolders
test_base_dir = './test_images/' # Directory containing 'Australasian_Bittern', ..., 'negative_images'

# 3. Name of the folder containing negative images (must match exactly)
negative_folder_name = "negative_images"

# 4. INFERENCE PARAMETERS
confidence_threshold = 0.55 # Adjust as needed - crucial for this evaluation
iou_threshold = 0.5       # IoU threshold for Non-Maximum Suppression (NMS)
device_to_use = 'cuda' if torch.cuda.is_available() else 'cpu'

# 5. Output control (optional, less relevant for this script)
# save_visualizations = False # Set to True to save annotated images in runs/detect/predict*
# output_dir_viz = './inference_results_eval/' # Only used if save_visualizations is True

# --- Initialization ---
print("--- Starting YOLOv8 Class Evaluation ---")
results_summary = defaultdict(lambda: {"correct": 0, "incorrect": 0, "missed": 0, "total": 0})
negative_summary = {"correct_neg": 0, "false_pos": 0, "total": 0}

# --- Checks ---
if not os.path.exists(model_path):
    print(f"ERROR: Model weights not found at: {model_path}")
    exit()
if not os.path.isdir(test_base_dir):
    print(f"ERROR: Test base directory not found at: {test_base_dir}")
    exit()

print(f"Loading model from: {model_path}")
print(f"Using device: {device_to_use}")
print(f"Confidence Threshold for evaluation: {confidence_threshold}")

try:
    # 1. Load the trained model
    model = YOLO(model_path)
    model.to(device_to_use)
    print("Model loaded successfully.")
    known_classes = list(model.names.values())
    print(f"Model knows {len(known_classes)} classes: {known_classes}")

    # 2. Iterate through class subdirectories
    class_folders = [d for d in os.listdir(test_base_dir) if os.path.isdir(os.path.join(test_base_dir, d))]
    print(f"\nFound {len(class_folders)} class folders to evaluate: {class_folders}")

    for ground_truth_class in class_folders:
        class_dir_path = os.path.join(test_base_dir, ground_truth_class)
        image_files = [f for f in os.listdir(class_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))]

        if not image_files:
            print(f"Skipping empty folder: {ground_truth_class}")
            continue

        print(f"\n--- Evaluating Class: {ground_truth_class} ({len(image_files)} images) ---")

        # Update total count for the class or negative summary
        if ground_truth_class == negative_folder_name:
             negative_summary["total"] = len(image_files)
        else:
             results_summary[ground_truth_class]["total"] = len(image_files)

        for img_file in image_files:
            img_path = os.path.join(class_dir_path, img_file)

            # Run prediction on the single image
            # Turn off saving unless specifically needed for debugging
            results = model.predict(
                source=img_path,
                conf=confidence_threshold,
                iou=iou_threshold,
                device=device_to_use,
                save=False, # Usually False for programmatic eval
                save_txt=False,
                verbose=False # Suppress per-image output
            )

            # Process results for this single image
            result = results[0] # Get the result object for the first (only) image
            boxes = result.boxes

            # --- Evaluation Logic ---
            is_negative_ground_truth = (ground_truth_class == negative_folder_name)

            if not is_negative_ground_truth:
                # --- Case 1: Ground Truth is a specific Bird Class ---
                if boxes.shape[0] == 0:
                    # No detection -> Missed (False Negative)
                    results_summary[ground_truth_class]["missed"] += 1
                    print(f"  MISS: No detection in {img_file}")
                else:
                    # Detections found -> Check the highest confidence prediction
                    best_pred_idx = boxes.conf.argmax()
                    pred_class_id = int(boxes.cls[best_pred_idx].item())
                    pred_class_name = model.names[pred_class_id]
                    pred_conf = boxes.conf[best_pred_idx].item()

                    if pred_class_name == ground_truth_class:
                        # Correct detection & classification
                        results_summary[ground_truth_class]["correct"] += 1
                        # print(f"  OK: Correctly found {pred_class_name} in {img_file} (Conf: {pred_conf:.2f})") # Optional detail
                    else:
                        # Incorrect classification (Misclassification)
                        results_summary[ground_truth_class]["incorrect"] += 1
                        print(f"  WRONG: Found {pred_class_name} instead of {ground_truth_class} in {img_file} (Conf: {pred_conf:.2f})")
            else:
                # --- Case 2: Ground Truth is Negative Image ---
                if boxes.shape[0] == 0:
                    # Correctly detected nothing
                    negative_summary["correct_neg"] += 1
                else:
                    # Incorrectly detected something (False Positive)
                    negative_summary["false_pos"] += 1
                    # Log what was detected (optional)
                    best_pred_idx = boxes.conf.argmax()
                    pred_class_id = int(boxes.cls[best_pred_idx].item())
                    pred_class_name = model.names[pred_class_id]
                    pred_conf = boxes.conf[best_pred_idx].item()
                    print(f"  FALSE POSITIVE: Detected {pred_class_name} in negative image {img_file} (Conf: {pred_conf:.2f})")

            # Optional: Add a small delay to prevent potential CUDA OOM on rapid calls, adjust if needed
            # time.sleep(0.01)


    # 3. Print Summary Report
    print("\n\n--- Evaluation Summary ---")
    print(f"Confidence Threshold used: {confidence_threshold}")

    # Print results for each positive class
    print("\nPositive Class Performance:")
    for class_name, counts in sorted(results_summary.items()):
        total = counts['total']
        if total == 0: continue
        correct = counts['correct']
        incorrect = counts['incorrect']
        missed = counts['missed']
        accuracy = (correct / total) * 100 if total > 0 else 0
        print(f"\n  Class: {class_name} ({total} images)")
        print(f"    - Correctly Classified: {correct} ({accuracy:.1f}%)")
        print(f"    - Misclassified As Other: {incorrect}")
        print(f"    - Missed (No Detection): {missed}")

    # Print results for the negative class
    print("\nNegative Image Performance:")
    neg_total = negative_summary['total']
    if neg_total == 0:
         print("  No negative images processed.")
    else:
        correct_neg = negative_summary['correct_neg']
        false_pos = negative_summary['false_pos']
        correct_neg_rate = (correct_neg / neg_total) * 100 if neg_total > 0 else 0
        fp_rate = (false_pos / neg_total) * 100 if neg_total > 0 else 0
        print(f"  Total Negative Images: {neg_total}")
        print(f"    - Correctly Ignored (No Detection): {correct_neg} ({correct_neg_rate:.1f}%)")
        print(f"    - False Positives (Detections Found): {false_pos} ({fp_rate:.1f}%)")

    print("\nEvaluation complete.")

except Exception as e:
    print(f"\nAn error occurred during evaluation: {e}")
    import traceback
    traceback.print_exc()

--- Starting YOLOv8 Class Evaluation ---
Loading model from: ./runs/detect/birds_320_python_run1_noprint10/weights/best.pt
Using device: cuda
Confidence Threshold for evaluation: 0.55
Model loaded successfully.
Model knows 10 classes: ['Australasian_Bittern', 'Australian_Painted_Snipe', 'Gang-gang_Cockatoo', 'Grey_Falcon', 'Painted_Honeyeater', 'Pilotbird', 'Plains_Wanderer', 'Southern_Whiteface', 'Swift_Parrot', 'White-throated_Needletail']

Found 11 class folders to evaluate: ['Australasian_Bittern', 'Australian_Painted_Snipe', 'Gang-gang_Cockatoo', 'Grey_Falcon', 'negative_images', 'Painted_Honeyeater', 'Pilotbird', 'Plains_Wanderer', 'Southern_Whiteface', 'Swift_Parrot', 'White-throated_Needletail']

--- Evaluating Class: Australasian_Bittern (5 images) ---
  WRONG: Found Australian_Painted_Snipe instead of Australasian_Bittern in 900.jpg (Conf: 0.94)
  MISS: No detection in IMG_0589-Bob-Green.jpg

--- Evaluating Class: Australian_Painted_Snipe (5 images) ---

--- Evaluating Class: